## 팡리 삭제

In [ ]:
import os

# XML 파일이 있는 폴더 경로
xml_folder = '/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images/rolled_pit'

# 폴더 내 XML 파일 삭제
def delete_xml_files(folder_path):
    # 폴더 내 파일 목록 가져오기
    files = os.listdir(folder_path)

    # XML 파일만 삭제
    for file in files:
        if file.endswith('.xml'):
            file_path = os.path.join(folder_path, file)
            os.remove(file_path)  # 파일 삭제
            print(f"Deleted: {file_path}")

# XML 파일 삭제 함수 실행
delete_xml_files(xml_folder)


In [5]:
import os
import glob
image_path = glob.glob('/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images/crease/*')
len(image_path)

262

## 좌우 상하 반전

In [8]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
결함 = ['crease', 'crescent_gap', 'inclusion','oil_spot','punching_hole','rolled_pit','silk_spot','waist folding', 'water_spot','welding_line']
for i in 결함:
    # 이미지 폴더 경로
    image_folder = f'C:\KDT_SF6\비전ai\archive\images\images\{i}'
    # XML 파일 경로
    xml_folder = 'C:\KDT_SF6\비전ai\archive\label\label/xml_single_bndbox'
    # 변경된 XML 파일 저장 경로 (새로 생성)
    xml_save_folder = 'C:\KDT_SF6\비전ai\archive\label\label/xml_single_bndbox'
    os.makedirs(xml_save_folder, exist_ok=True)

    # 이미지 파일 목록 가져오기
    image_files = os.listdir(image_folder)
    xml_files = os.listdir(xml_folder)

    # 경로와 확장자를 제외한 파일 이름 추출 함수
    def extract_filename_without_extension(file_path):
        # 경로를 '/'로 분절한 후, 마지막 요소(파일명)를 선택
        filename = file_path.split('/')[-1]
        # 파일명에서 확장자를 제외하고 반환
        return filename.split('.')[0]

    # 뒤집기 접미사 설정
    flip_suffixes = {
        "left_right": '_flipped_lr',
        "top_bottom": '_flipped_tb'
    }

    # 이미지 뒤집기 및 XML 좌표 업데이트 함수
    def flip_image_and_update_xml(image_file, flip_type):
        # 이미지 경로 설정
        image_path = os.path.join(image_folder, image_file)
        img = Image.open(image_path)
        img_width, img_height = img.size

        # 이미지 뒤집기
        if flip_type == "left_right":
            flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
        elif flip_type == "top_bottom":
            flipped_img = img.transpose(Image.FLIP_TOP_BOTTOM)

        # 이미지 저장 경로 설정 (여기서는 이미지 저장 폴더를 그대로 두고, XML만 이동)
        flipped_image_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.jpg")
        save_path = os.path.join(image_folder, flipped_image_filename)
        flipped_img.save(save_path)
        print(f"Flipped image saved at: {save_path}")

        # XML 파일 로드
        xml_file = os.path.join(xml_folder, image_file.replace('.jpg', '.xml'))
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # 바운딩 박스 정보 업데이트
        for obj in root.findall('object'):
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # 좌우 뒤집기 (좌우 반전)
            if flip_type == "left_right":
                new_xmin = img_width - xmax
                new_xmax = img_width - xmin
                new_ymin = ymin
                new_ymax = ymax
            # 상하 뒤집기 (상하 반전)
            elif flip_type == "top_bottom":
                new_xmin = xmin
                new_xmax = xmax
                new_ymin = img_height - ymax
                new_ymax = img_height - ymin

            # 업데이트된 좌표로 XML 수정
            bndbox.find('xmin').text = str(new_xmin)
            bndbox.find('ymin').text = str(new_ymin)
            bndbox.find('xmax').text = str(new_xmax)
            bndbox.find('ymax').text = str(new_ymax)

        # 변경된 XML을 새로운 경로에 저장
        flipped_xml_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.xml")
        flipped_xml_path = os.path.join(xml_save_folder, flipped_xml_filename)
        tree.write(flipped_xml_path)
        print(f"Flipped XML saved at: {flipped_xml_path}")

    # 모든 이미지 파일에 대해 좌우 및 상하 뒤집기 수행
    for image_file in image_files:
        # 이미지 파일명에서 확장자를 제외한 이름 추출
        image_filename = extract_filename_without_extension(image_file)

        # XML 파일 중 동일한 이름이 있는지 확인
        xml_filename_matches = [xml_file for xml_file in xml_files if extract_filename_without_extension(xml_file) == image_filename]

        if xml_filename_matches:
            for flip in ["left_right", "top_bottom"]:
                flip_image_and_update_xml(image_file, flip)
        else:
            print(f"Skipping {image_file}: corresponding XML file not found.")


OSError: [WinError 123] 파일 이름, 디렉터리 이름 또는 볼륨 레이블 구문이 잘못되었습니다: 'C:\\KDT_SF6\\비전ai\x07rchive'

In [ ]:
import os
from PIL import Image
import xml.etree.ElementTree as ET

# 이미지 파일과 XML 파일 경로를 직접 지정
image_file = '/content/drive/MyDrive/비전ai/archive/images/images/images/crease/example_image.jpg'
xml_file = '/content/drive/MyDrive/비전ai/archive/label/xml_single_bndbox/example_image.xml'

# 회전한 이미지를 저장할 폴더 경로 (이미지 파일이 속한 폴더로 지정)
rotated_image_folder = os.path.dirname(image_file)
os.makedirs(rotated_image_folder, exist_ok=True)

# 각도를 기준으로 파일명 접미사 설정
rotation_suffixes = {
    90: '_1',
    180: '_2',
    270: '_3'
}

# 이미지 회전 및 XML 좌표 업데이트 함수
def rotate_image_and_update_xml(image_file, xml_file, rotation_angle):
    # 이미지 경로 설정
    img = Image.open(image_file)
    img_width, img_height = img.size

    # 이미지 회전
    rotated_img = img.rotate(rotation_angle, expand=True)

    # 이미지 저장 경로 설정
    rotated_image_filename = os.path.basename(image_file).replace('.jpg', f"{rotation_suffixes[rotation_angle]}.jpg")
    save_path = os.path.join(rotated_image_folder, rotated_image_filename)
    rotated_img.save(save_path)
    print(f"Rotated image saved at: {save_path}")

    # XML 파일 로드
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 바운딩 박스 정보 업데이트
    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # 회전 각도에 따른 바운딩 박스 좌표 변환
        if rotation_angle == 90:
            new_xmin = ymin
            new_ymin = img_width - xmax
            new_xmax = ymax
            new_ymax = img_width - xmin
        elif rotation_angle == 180:
            new_xmin = img_width - xmax
            new_ymin = img_height - ymax
            new_xmax = img_width - xmin
            new_ymax = img_height - ymin
        elif rotation_angle == 270:
            new_xmin = img_height - ymax
            new_ymin = xmin
            new_xmax = img_height - ymin
            new_ymax = xmax

        # 업데이트된 좌표로 XML 수정
        bndbox.find('xmin').text = str(new_xmin)
        bndbox.find('ymin').text = str(new_ymin)
        bndbox.find('xmax').text = str(new_xmax)
        bndbox.find('ymax').text = str(new_ymax)

    # 회전된 이미지에 맞는 XML 저장
    rotated_xml_filename = os.path.basename(xml_file).replace('.xml', f"{rotation_suffixes[rotation_angle]}.xml")
    rotated_xml_path = os.path.join(rotated_image_folder, rotated_xml_filename)
    tree.write(rotated_xml_path)
    print(f"Rotated XML saved at: {rotated_xml_path}")

# 특정 각도로 회전 수행 (예: 90도, 180도, 270도)
for angle in [90, 180, 270]:
    rotate_image_and_update_xml(image_file, xml_file, angle)


In [5]:
import os
from PIL import Image
import xml.etree.ElementTree as ET

# 결함 리스트
결함 = ['crease', 'crescent_gap', 'inclusion', 'oil_spot', 'punching_hole', 'rolled_pit', 'silk_spot', 'waist folding', 'water_spot', 'welding_line']

for 결함_항목 in 결함:
    # 이미지 폴더 경로
    image_folder = 'C:/KDT_SF6/비전ai/archive/images/images/crease'
    # XML 파일 경로
    xml_folder = 'C:/KDT_SF6/비전ai/archive/label/label/xml_single_bndbox'
    # 변경된 XML 파일 저장 경로 (새로 생성)
    xml_save_folder = 'C:/KDT_SF6\\비전ai\\archive\\label\\label\\xml_single_bndbox'
    os.makedirs(xml_save_folder, exist_ok=True)

    # 이미지 파일 목록 가져오기
    image_files = os.listdir(image_folder)
    xml_files = os.listdir(xml_folder)

    # 경로와 확장자를 제외한 파일 이름 추출 함수
    def extract_filename_without_extension(file_path):
        # 경로를 '\\'로 분절한 후, 마지막 요소(파일명)를 선택
        filename = file_path.split('\\')[-1]
        # 파일명에서 확장자를 제외하고 반환
        return filename.split('.')[0]

    # 뒤집기 접미사 설정
    flip_suffixes = {
        "left_right": '_flipped_lr',
        "top_bottom": '_flipped_tb'
    }

    # 이미지 뒤집기 및 XML 좌표 업데이트 함수
    def flip_image_and_update_xml(image_file, flip_type):
        # 이미지 경로 설정
        image_path = os.path.join(image_folder, image_file)
        img = Image.open(image_path)
        img_width, img_height = img.size

        # 이미지 뒤집기
        if flip_type == "left_right":
            flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
        elif flip_type == "top_bottom":
            flipped_img = img.transpose(Image.FLIP_TOP_BOTTOM)

        # 이미지 저장 경로 설정 (여기서는 이미지 저장 폴더를 그대로 두고, XML만 이동)
        flipped_image_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.jpg")
        save_path = os.path.join(image_folder, flipped_image_filename)
        flipped_img.save(save_path)
        print(f"Flipped image saved at: {save_path}")

        # XML 파일 로드
        xml_file = os.path.join(xml_folder, image_file.replace('.jpg', '.xml'))
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # 바운딩 박스 정보 업데이트
        for obj in root.findall('object'):
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # 좌우 뒤집기 (좌우 반전)
            if flip_type == "left_right":
                new_xmin = img_width - xmax
                new_xmax = img_width - xmin
                new_ymin = ymin
                new_ymax = ymax
            # 상하 뒤집기 (상하 반전)
            elif flip_type == "top_bottom":
                new_xmin = xmin
                new_xmax = xmax
                new_ymin = img_height - ymax
                new_ymax = img_height - ymin

            # 업데이트된 좌표로 XML 수정
            bndbox.find('xmin').text = str(new_xmin)
            bndbox.find('ymin').text = str(new_ymin)
            bndbox.find('xmax').text = str(new_xmax)
            bndbox.find('ymax').text = str(new_ymax)

        # 변경된 XML을 새로운 경로에 저장
        flipped_xml_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.xml")
        flipped_xml_path = os.path.join(xml_save_folder, flipped_xml_filename)
        tree.write(flipped_xml_path)
        print(f"Flipped XML saved at: {flipped_xml_path}")

    # 모든 이미지 파일에 대해 좌우 및 상하 뒤집기 수행
    for image_file in image_files:
        # 이미지 파일명에서 확장자를 제외한 이름 추출
        image_filename = extract_filename_without_extension(image_file)

        # XML 파일 중 동일한 이름이 있는지 확인
        xml_filename_matches = [xml_file for xml_file in xml_files if extract_filename_without_extension(xml_file) == image_filename]

        if xml_filename_matches:
            for flip in ["left_right", "top_bottom"]:
                flip_image_and_update_xml(image_file, flip)
        else:
            print(f"Skipping {image_file}: corresponding XML file not found.")


Skipping img_01_3436789500_00004.jpg: corresponding XML file not found.
Skipping img_01_425382900_00002.jpg: corresponding XML file not found.
Skipping img_01_425501700_00022.jpg: corresponding XML file not found.
Skipping img_01_429539000_00002.jpg: corresponding XML file not found.
Skipping img_01_4402116900_00003.jpg: corresponding XML file not found.
Skipping img_01_4402117100_00006.jpg: corresponding XML file not found.
Skipping img_01_4402117200_00003.jpg: corresponding XML file not found.
Skipping img_01_4402270500_00003.jpg: corresponding XML file not found.
Skipping img_01_4402270500_00004.jpg: corresponding XML file not found.
Skipping img_01_4403690400_00005.jpg: corresponding XML file not found.
Skipping img_02_3436789500_00005.jpg: corresponding XML file not found.
Skipping img_02_425613700_00758.jpg: corresponding XML file not found.
Skipping img_02_4402116700_00002.jpg: corresponding XML file not found.
Skipping img_02_4406430100_00002.jpg: corresponding XML file not fou

## 회전

In [7]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
결함 = ['crease', 'crescent_gab', 'inclusion','oil_spot','punching_hole','rolled_pit','silk_spot','waist folding', 'water_spot','welding_line']
for i in 결함:
    # 이미지 폴더 경로
    image_folder = f'C:/KDT_SF6/비전ai/archive/images/images/{i}'
    # 회전한 이미지를 저장할 폴더 경로 (새로 생성)
    rotated_image_folder = f'C:/KDT_SF6/비전ai/archive/images/images/{i}'
    os.makedirs(rotated_image_folder, exist_ok=True)

    # XML 파일 경로
    xml_folder = 'C:/KDT_SF6\비전ai/archive/label/label/xml_single_bndbox'

    # 이미지 파일 목록 가져오기
    image_files = os.listdir(image_folder)
    xml_files = os.listdir(xml_folder)

    # 각도를 기준으로 파일명 접미사 설정
    rotation_suffixes = {
        90: '_1',
        180: '_2',
        270: '_3'
    }

    # 이미지 회전 및 XML 좌표 업데이트 함수
    def rotate_image_and_update_xml(image_file, rotation_angle):
        # 이미지 경로 설정
        image_path = os.path.join(image_folder, image_file)
        img = Image.open(image_path)
        img_width, img_height = img.size

        # 이미지 회전
        rotated_img = img.rotate(rotation_angle, expand=True)

        # 이미지 저장 경로 설정
        rotated_image_filename = image_file.replace('.jpg', f"{rotation_suffixes[rotation_angle]}.jpg")
        save_path = os.path.join(rotated_image_folder, rotated_image_filename)
        rotated_img.save(save_path)
        print(f"Rotated image saved at: {save_path}")

        # XML 파일 로드
        xml_file = os.path.join(xml_folder, image_file.replace('.jpg', '.xml'))
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # 바운딩 박스 정보 업데이트
        for obj in root.findall('object'):
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # 회전 각도에 따른 바운딩 박스 좌표 변환
            if rotation_angle == 90:
                new_xmin = img_height - ymax
                new_ymin = xmin
                new_xmax = img_height - ymin
                new_ymax = xmax
            elif rotation_angle == 180:
                new_xmin = img_width - xmax
                new_ymin = img_height - ymax
                new_xmax = img_width - xmin
                new_ymax = img_height - ymin
            elif rotation_angle == 270:
                new_xmin = ymin
                new_ymin = img_width - xmax
                new_xmax = ymax
                new_ymax = img_width - xmin

            # 업데이트된 좌표로 XML 수정
            bndbox.find('xmin').text = str(new_xmin)
            bndbox.find('ymin').text = str(new_ymin)
            bndbox.find('xmax').text = str(new_xmax)
            bndbox.find('ymax').text = str(new_ymax)

        # 회전된 이미지에 맞는 XML 저장
        rotated_xml_filename = image_file.replace('.jpg', f"{rotation_suffixes[rotation_angle]}.xml")
        rotated_xml_path = os.path.join(rotated_image_folder, rotated_xml_filename)
        tree.write(rotated_xml_path)
        print(f"Rotated XML saved at: {rotated_xml_path}")

    # 모든 이미지 파일에 대해 90도, 180도, 270도 회전 수행
    for image_file in image_files:
        # XML 파일이 존재하는지 확인
        corresponding_xml = image_file.replace('.jpg', '.xml')
        if corresponding_xml in xml_files:
            for angle in [90, 180, 270]:
                rotate_image_and_update_xml(image_file, angle)
        else:
            print(f"Skipping {image_file}: corresponding XML file not found.")


Skipping img_01_3436789500_00004.jpg: corresponding XML file not found.
Skipping img_01_425382900_00002.jpg: corresponding XML file not found.
Skipping img_01_425501700_00022.jpg: corresponding XML file not found.
Skipping img_01_429539000_00002.jpg: corresponding XML file not found.
Skipping img_01_4402116900_00003.jpg: corresponding XML file not found.
Skipping img_01_4402117100_00006.jpg: corresponding XML file not found.
Skipping img_01_4402117200_00003.jpg: corresponding XML file not found.
Skipping img_01_4402270500_00003.jpg: corresponding XML file not found.
Skipping img_01_4402270500_00004.jpg: corresponding XML file not found.
Skipping img_01_4403690400_00005.jpg: corresponding XML file not found.
Skipping img_02_3436789500_00005.jpg: corresponding XML file not found.
Skipping img_02_425613700_00758.jpg: corresponding XML file not found.
Skipping img_02_4402116700_00002.jpg: corresponding XML file not found.
Skipping img_02_4406430100_00002.jpg: corresponding XML file not fou

In [9]:
# import os
# from PIL import Image
# import xml.etree.ElementTree as ET

# # 이미지 폴더 경로
# image_folder = '/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images/punching_hole'
# # XML 파일 경로
# xml_folder = '/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/label/xml_single_bndbox'
# # 변경된 XML 파일 저장 경로 (새로 생성)
# xml_save_folder = '/content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/label/xml_single_bndbox'
# os.makedirs(xml_save_folder, exist_ok=True)

# # 이미지 파일 목록 가져오기
# image_files = os.listdir(image_folder)
# xml_files = os.listdir(xml_folder)

# # 경로와 확장자를 제외한 파일 이름 추출 함수
# def extract_filename_without_extension(file_path):
#     # 경로를 '/'로 분절한 후, 마지막 요소(파일명)를 선택
#     filename = file_path.split('/')[-1]
#     # 파일명에서 확장자를 제외하고 반환
#     return filename.split('.')[0]

# # 뒤집기 접미사 설정
# flip_suffixes = {
#     "left_right": '_flipped_lr',
#     "top_bottom": '_flipped_tb'
# }

# # 이미지 뒤집기 및 XML 좌표 업데이트 함수
# def flip_image_and_update_xml(image_file, flip_type):
#     # 이미지 경로 설정
#     image_path = os.path.join(image_folder, image_file)
#     img = Image.open(image_path)
#     img_width, img_height = img.size

#     # 이미지 뒤집기
#     if flip_type == "left_right":
#         flipped_img = img.transpose(Image.FLIP_LEFT_RIGHT)
#     elif flip_type == "top_bottom":
#         flipped_img = img.transpose(Image.FLIP_TOP_BOTTOM)

#     # 이미지 저장 경로 설정 (여기서는 이미지 저장 폴더를 그대로 두고, XML만 이동)
#     flipped_image_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.jpg")
#     save_path = os.path.join(image_folder, flipped_image_filename)
#     flipped_img.save(save_path)
#     print(f"Flipped image saved at: {save_path}")

#     # XML 파일 로드
#     xml_file = os.path.join(xml_folder, image_file.replace('.jpg', '.xml'))
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     # 바운딩 박스 정보 업데이트
#     for obj in root.findall('object'):
#         bndbox = obj.find('bndbox')
#         xmin = int(bndbox.find('xmin').text)
#         ymin = int(bndbox.find('ymin').text)
#         xmax = int(bndbox.find('xmax').text)
#         ymax = int(bndbox.find('ymax').text)

#         # 좌우 뒤집기 (좌우 반전)
#         if flip_type == "left_right":
#             new_xmin = img_width - xmax
#             new_xmax = img_width - xmin
#             new_ymin = ymin
#             new_ymax = ymax
#         # 상하 뒤집기 (상하 반전)
#         elif flip_type == "top_bottom":
#             new_xmin = xmin
#             new_xmax = xmax
#             new_ymin = img_height - ymax
#             new_ymax = img_height - ymin

#         # 업데이트된 좌표로 XML 수정
#         bndbox.find('xmin').text = str(new_xmin)
#         bndbox.find('ymin').text = str(new_ymin)
#         bndbox.find('xmax').text = str(new_xmax)
#         bndbox.find('ymax').text = str(new_ymax)

#     # 변경된 XML을 새로운 경로에 저장
#     flipped_xml_filename = image_file.replace('.jpg', f"{flip_suffixes[flip_type]}.xml")
#     flipped_xml_path = os.path.join(xml_save_folder, flipped_xml_filename)
#     tree.write(flipped_xml_path)
#     print(f"Flipped XML saved at: {flipped_xml_path}")

# # 모든 이미지 파일에 대해 좌우 및 상하 뒤집기 수행
# for image_file in image_files:
#     # 이미지 파일명에서 확장자를 제외한 이름 추출
#     image_filename = extract_filename_without_extension(image_file)

#     # XML 파일 중 동일한 이름이 있는지 확인
#     xml_filename_matches = [xml_file for xml_file in xml_files if extract_filename_without_extension(xml_file) == image_filename]

#     if xml_filename_matches:
#         for flip in ["left_right", "top_bottom"]:
#             flip_image_and_update_xml(image_file, flip)
#     else:
#         print(f"Skipping {image_file}: corresponding XML file not found.")


Flipped image saved at: /content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images/punching_hole/img_02_425640100_00874_flipped_lr.jpg
Flipped XML saved at: /content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/label/xml_single_bndbox/img_02_425640100_00874_flipped_lr.xml
Flipped image saved at: /content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/images/images/punching_hole/img_02_425640100_00874_flipped_tb.jpg
Flipped XML saved at: /content/drive/MyDrive/Colab Notebooks/ai프로젝트/input/label/xml_single_bndbox/img_02_425640100_00874_flipped_tb.xml
Skipping img_03_3403397500_01395.jpg: corresponding XML file not found.
Skipping img_03_3403395700_01033.jpg: corresponding XML file not found.
Skipping img_02_425616500_00001.jpg: corresponding XML file not found.
Skipping img_03_3403394100_01319.jpg: corresponding XML file not found.
Skipping img_03_3403400500_00340.jpg: corresponding XML file not found.
Flipped image saved at: /content/drive/MyDrive/Colab Notebooks/ai프로젝트/